In [28]:
from typing import Tuple
!!auto-py-to-exe

^C


[]

In [35]:
# import pandas as pd
# gpa = pd.read_csv('sfrbx_gps_alm.csv', index_col=0)
# gpe = pd.read_csv('sfrbx_gps_eph.csv', index_col=0)
# gla = pd.read_csv('sfrbx_glonass_alm.csv', index_col=0)
# gle = pd.read_csv('sfrbx_glonass_eph.csv', index_col=0)
# with open('sfrbx_gps_data.txt', 'r') as file:
#     gp_data = eval(file.read())

In [34]:
# import pickle
# from datetime import datetime
# import Constants
# t1 = datetime.now(tz=Constants.tz_utc)
# SFRBX_GLONASS_PATH = 'glonass_sfrbx.plk'
# with open(SFRBX_GLONASS_PATH, 'wb') as file:
#     pickle.dump(gpe, file)
#     pickle.dump(gpa, file)
#     pickle.dump(gle, file)
#     pickle.dump(gla, file)
#     pickle.dump(gp_data, file)
# t2 = datetime.now(tz=Constants.tz_utc)
# print(t2-t1)  

0:00:00.002186


In [68]:
# SFRBX_GLONASS_PATH = 'glonass_sfrbx.plk'
SFRBX_path = 'sfrbx_data.plk'
import pickle
with open(SFRBX_path, 'rb') as file:
    gpe7 = pickle.load(file)
    gpa7 = pickle.load(file)
    gle7 = pickle.load(file)
    gla7 = pickle.load(file)
    gpD7 = pickle.load(file)
# gla1 == gla

In [19]:
import numpy as np
from math import pi, floor, sin, cos, atan, sqrt, tan, atan2
import Constants
import Transformations


def get_glo_year(N4, N):
    if 1 <= N <= 366:
        J=1
    elif 367 <= N <= 731:
        J = 2
    elif 732 <= N <= 1096:
        J = 3
    elif 1097<= N <= 1461:
        J = 4
    else:
        J = 0 # TODO - error
    return 1996 + 4 * (N4-1) + (J-1)

def get_glo_dNA(N4, NA, N):
    k = 1461 if N4 != 27 else 1460
    return N - NA - round((N-NA)/k)*k

#TODO: переименовать N в NA в преобраовании ГЛОНАСС 
# t1 = 81300 # 22:35
t1 = 24 * 3600 + 0 * 60
print(f'{t1//3600}h : {(t1%3600//60)}m = ({t1} sec)')
N = 204 # 22.07.2024
# t1, N - время и день, на которые проводится расчет

def calc_glo_alm(alm, eph, t1, N, hard=True):
    GM = Constants.mu
    ae = Constants.ae_glonass
    J20 = Constants.J20_glonass
    OmegaEath = Constants.OmegaEarthDot
    
    NA = eph.NA
    N4 = eph.N4
    t_lambda = alm.t_lambda_n
    dT = alm.delta_T_n
    di = alm.delta_i_n
    dT_dot = alm.delta_T_dot_n
    e = alm.eps_n
    w_A = alm.omega_n
    lambda_A = alm.lambda_n
    
    # для проверки
    # N = 1453
    # t1 = 51300
    # NA = 1452
    # t_lambda = 33571.625
    # dT = 0.01953124999975
    # dT_dot= 6.103515625E-05
    # lambda_A = -0.293967247009277 * pi
    # w_A = 0.57867431640625 * pi
    # e = 0.000432968139648438
    # di = -0.00012947082519531 * pi
    
    # 1 - интервал прогноза dt_pr в сек
    dNa = get_glo_dNA(N4=N4, NA=NA, N=N)
    dt_pr = dNa * 86400 + (t1 - t_lambda)
    # print(1, dt_pr)
    
    # 2 - количество целых витков W на интервале прогноза
    Tdr_avg = 43200
    W = floor(dt_pr / (Tdr_avg + dT))
    # print(2, W)
    
    # 3 - текущее наклонение
    i_avg = 63 / 180 * pi
    i = di + i_avg
    # print(3, i)
    #TODO: результат не сходится с примеров в документации
    # #i = 1.1305666106990377
    
    # 4 - средний драконический период на витке W+1 и среднее движение
    Tdr = Tdr_avg + dT + (2*W + 1) * dT_dot
    n = 2*pi / Tdr
    # print(4, Tdr, n)
    
    # 5 - рассчитывается большая полуось последовательными приближениями (m=0, 1, 2...)
    T1 = Tdr
    a0 = -1
    a = 0
    p = 0
    while abs(a0-a) > 1e-3:
        a0 = a
        a = ((T1/2/pi)**2 * GM) ** (1.0/3.0)
        p = a * (1-e**2)
        Ttt = (2-2.5*sin(i)**2) * ((1-e**2) ** 1.5 / (1 + e * cos(w_A * pi)) ** 2) + (1 + e * cos(w_A * pi)) ** 3 / (1 - e ** 2)
        Tdown = 1 - 1.5 * J20 * (ae/p)** 2 * Ttt
        T1 = Tdr / Tdown
    # print(5, a1, p1, T1)
    
    # 6 - текущее значение долготы восходящего узла и аргумента перигея
    lmb = lambda_A - (OmegaEath + 1.5*J20*n*(ae/p)**2*cos(i))* dt_pr
    w = w_A - 0.75 * J20 * n * (ae/p)**2 * (1-5*cos(i)**2) * dt_pr
    # print(6, lmb, w)
    
    # 7 - среднее значение долготы на момент прохождения восходящего узла
    E0 = -2 * atan(sqrt((1-e)/(1+e)) * tan(w/2))
    L1 = w + E0 - e*sin(E0)
    # print(7, E0, L1)
    
    # 8 - текущее значение долготы НКА
    L = L1 + n * (dt_pr - (Tdr_avg + dT)*W - dT_dot*W**2)
    L2 = L
    # print(8, L)
    
    # 9 - для повышения точности, можно опустить
    def calc_derivatives(Lk):
        B = 1.5 * J20 * (ae/a)**2
        da_a_k = 2*B*(1-1.5*sin(i)**2) * (l*cos(Lk) + h * sin(Lk)) + B * sin(i)**2 * (0.5*h*sin(Lk) - 0.5*l*cos(Lk) + cos(2*Lk) + 3.5*l*cos(3*Lk) + 3.5*h*sin(3*Lk))
        da_k = da_a_k * a
        dh_k = B * (1-1.5*sin(i)**2) * (sin(Lk)+1.5*l*sin(2*Lk)-1.5*h*cos(2*Lk)) - 0.25*B*sin(i)**2 * (sin(Lk)-(7.0/3.0) * sin(3*Lk) + 5*l*sin(2*Lk) - 8.5*l*sin(4*Lk) + 8.5*h*cos(4*Lk) + h*cos(2*Lk)) + (-0.5*B*cos(i)**2*l*sin(2*Lk))
        dl_k = B * (1-1.5*sin(i)**2) * (cos(Lk)+1.5*l*cos(2*Lk)+1.5*h*sin(2*Lk)) - 0.25*B*sin(i)**2 * (-cos(Lk)-(7.0/3.0)* cos(3*Lk) - 5*h*sin(2*Lk) - 8.5*l*cos(4*Lk) - 8.5*h*sin(4*Lk) + l*cos(2*Lk)) + 0.5*B*cos(i)**2*h*sin(2*Lk)
        dlmb_k = -B*cos(i)*(3.5*l*sin(Lk)-2.5*h*cos(Lk)-0.5*sin(2*Lk)-(7.0/6.0)*l*sin(3*Lk)+(7.0/6.0)*h*cos(3*Lk))
        di_k = 0.5*B*sin(i)*cos(i)*(-l*cos(Lk)+h*sin(Lk) +cos(2*Lk) + (7.0/3.0)*l*cos(3*Lk)+(7.0/3.0)*h*sin(3*Lk))
        dL_k = 2*B*(1-1.5*sin(i)**2) * (1.75*l*sin(Lk) - 1.75*h*cos(Lk)) + 3*B*sin(i)**2 * (-(7.0/24.0)*h*cos(Lk)-(7.0/24.0)*l*sin(Lk) -(49.0/72.0)*h*cos(3*Lk) + (49.0/72.0)*l*sin(3*Lk) + 0.25*sin(2*Lk)) + B*cos(i)**2 * (3.5*l*sin(Lk)-2.5*h*cos(Lk)-0.5*sin(2*Lk)-(7.0/6.0)*l*sin(3*Lk) + (7.0/6.0)*h*cos(3*Lk))
        return da_k, dh_k, dl_k, dlmb_k, di_k, dL_k
    if hard:
        h = e * sin(w)
        l = e * cos(w)
        
        da1, dh1, dl1, dlmb1, di1, dL1 = calc_derivatives(Lk=L1)
        da2, dh2, dl2, dlmb2, di2, dL2 = calc_derivatives(Lk=L2)
        
        a = a + da2 - da1
        h = h + dh2 - dh1
        l = l + dl2 - dl1
        e = sqrt(h**2+l**2)
        i = i + di2 - di1
        lmb = lmb + dlmb2 - dlmb1
        w = atan2(h, l) #atan(h/l)
        L = L + dL2 - dL1
    
    # 10 - эксцентрическая аномалия
    E1 = 10 + E0
    while abs(E1-E0) > 1e-9:
        E0 = E1
        E1 = L - w + e * sin(E0)
    E = E1
    
    # 11 истинняа аномалия и аргумент широты
    nu = 2 * atan( sqrt((1+e)/(1-e)) * tan(E/2))
    u = nu + w
    
    # 12 - координаты ЦМ в прямоугольной ск
    p = a * (1-e**2)
    r = p / (1+e * cos(nu))
    X = r * (cos(lmb)*cos(u)-sin(lmb)*sin(u)*cos(i))
    Y = r * (sin(lmb)*cos(u)+cos(lmb)*sin(u)*cos(i))
    Z = r * sin(u) * sin(i)
    
    # Vr = sqrt(GM/p) * e * sin(nu)
    # Vu = sqrt(GM/p) * (1+e*cos(nu))
    # Vx = Vr * (cos(lmb)*cos(u)-sin(lmb)*sin(u)*cos(i)) - Vu * (cos(lmb)*sin(u)+sin(lmb)*cos(u)*cos(i)) + OmegaEath * Y
    # Vy = Vr * (sin(lmb)*cos(u)+cos(lmb)*sin(u)*cos(i)) - Vu * (sin(lmb)*sin(u)-cos(lmb)*cos(u)*cos(i)) - OmegaEath * X
    # Vz = Vr * (sin(u)*sin(i)) + Vu * (cos(u)*sin(i))
    
    return X, Y, Z

prMes = {
    7: 17931698.162640072,
    16: 17247315.440587413,
    17: 19352936.817768987,
    24: 21353624.2500098,
}
# import Transformations
# data = []
# eph = gle.loc[6]
# for svId in prMes.keys():
#     alm = gla.loc[svId-1]
#     # eph = gle.loc[svId-1]
#     assert svId == alm.svId# and svId == eph.svId
#     x, y, z = calc_glo_alm(alm, eph, t1, N)
#     print(svId, tuple(np.array(Transformations.ecef2lla(x, y, z)).round(2)), round(prMes[svId], -3))
#     data.append((x, y, z, prMes[svId]))

24h : 0m = (86400 sec)


In [67]:
sv = 4
eph = gle3.loc[sv-1]
eph2 = gle5.loc[sv-1]
# eph3 = gle4.loc[sv-1]
alm = gla3.loc[sv-1]
alm2 = gla5.loc[sv-1]
N = 205
t1 = eph2.tb
# xa, ya, za = calc_glo_alm(alm, eph, t1, N, hard=True)
xa, ya, za = calc_glo_alm(gla4.loc[sv-1], eph, t1, N, hard=True)
# xa2, ya2, za2 = calc_glo_alm(alm2, eph2, t1, N, hard=True)
xe, ye, ze, vx, vy, vz = calc_glo_eph_simple(eph, t1, N).y[:, -1]
x2, y2, z2, vx2, vy2, vz2 = eph2[['x', 'y', 'z', 'dx', 'dy', 'dz']] * 1000

print(xa, ya, za, Transformations.ecef2lla(xa, ya, za), 'alm calced')
# print(xa2, ya2, za2, Transformations.ecef2lla(xa2, ya2, za2))
print(xe, ye, ze, vx, vy, vz, 'eph calced')
print(x2, y2, z2, vx2, vy2, vz2, 'from eph')
# print(eph3[['x', 'y', 'z', 'dx', 'dy', 'dz']])

2542402.4373395117 25369517.467169758 -463051.8096480523 (-1.0421973217533904, 84.2772188935492, 19122666.97288603) alm calced
2544529.9122669343 25369228.130712498 -460934.54010835016 193.9920737529572 -88.41871007550772 -3589.5457736995468 eph calced
2544520.99609375 25369229.98046875 -460937.98828125 193.98212432861328 -88.41609954833984 -3589.5490646362305 from eph


In [66]:
import numpy as np
from math import pi, floor, sin, cos, atan, sqrt, tan, atan2
import Constants
import Transformations
from scipy.integrate import RK45 , solve_ivp, odeint

def calc_glo_eph_simple(eph, t1, N):
    X0 = eph.x
    Vx0 = eph.dx
    ddx = eph.ddx
    Y0 = eph.y
    Vy0 = eph.dy
    ddy = eph.ddy
    Z0 = eph.z
    Vz0 = eph.dz
    ddz = eph.ddz
    tb = eph.tb
    
    maxT = lambda ti: ti - round((ti-tb)/86400)*86400
    
    ae = Constants.ae_glonass
    GM = Constants.mu
    J20 = Constants.J20_glonass
    OmegaEarth = Constants.OmegaEarthDot
    
    y0 = np.array([X0, Y0, Z0, Vx0, Vy0, Vz0]) * 1000
    def func(t, y):
        X, Y, Z, Vx, Vy, Vz = y
        r = sqrt(X**2+Y**2+Z**2)
        dVx = -GM/r**3 * X - 1.5*J20*GM*ae**2/r**5 * X * (1 - 5*Z**2/r**2) + OmegaEarth**2 * X + 2 * OmegaEarth * Vy + ddx
        dVy = -GM/r**3 * Y - 1.5*J20*GM*ae**2/r**5 * Y * (1 - 5*Z**2/r**2) + OmegaEarth**2 * Y - 2 * OmegaEarth * Vx + ddy
        dVz = -GM/r**3 * Z - 1.5*J20*GM*ae**2/r**5 * Z * (3 - 5*Z**2/r**2) + ddz
        # print(t, np.array([X, Y, Z, Vx, Vy, Vz, dVx, dVy, dVz]).round(0))
        return np.array([Vx, Vy, Vz, dVx, dVy, dVz])
    # runer = RK45(func, tb, y0, t1, first_step=1)
    # 
    # while runer.status == 'running':
    #     runer.step()
    #     # print(runer.t, runer.y.round(3))
    # # print(runer.t, runer.y.round(3))
    # return runer.y
    res = solve_ivp(func, (tb, t1), y0, method='RK45', max_step=10, first_step=1)
    return res

# t1 = 81300 # 22:35
# t1 = 23 * 3600 + 43 * 60
print(f'{t1//3600}h : {(t1%3600//60)}min')
N = 204 # 22.07.2024
# t1, N - время и день, на которые проводится расчет
res = calc_glo_eph_simple(eph, t1, N)
x, y, z, Vx, Vy, Vz = res.y[:, -1]
print(eph.svId)
print(x, y, z)
print(Transformations.ecef2lla(x, y, z))

19.0h : 45.0m
4
2544529.9122669343 25369228.130712498 -460934.54010835016
(-1.0374360825490556, 84.27239702447706, 19122552.900628805)


In [62]:
def calc_julian_day(N4: int, N: int) -> tuple[int, int, int, int, float]:
    JD0 = 1461 * (N4-1) + N + 2450082.5
    JDN = JD0 + 0.5
    a = JDN + 32044
    b = (4*a+3)//146097
    c = a - (146097*b)//4
    d = (4*c+3) // 1461
    e =c - (1461*d)//4
    m = (5*e+2)//153
    
    day: int = e - (153*m+2) // 5 + 1
    month: int = m + 3 - 12 * (m//10)
    year: int = 100*b + d - 4800 + (m//10)
    day_of_week: int = JDN % 7 + 1
    
    ERA = 2 * pi * (0.7790572732640 + 1.00273781191135448 * (JD0 - 2451545.0))
    dT = (JD0 - 2451545.0) / 36525
    GMST = ERA + 0.0000000703270726 + 0.0223603658710194 * dT + 0.0000067465784654 * dT**2  - 0.0000000000021332 * dT**3  -0.0000000001452308 *dT**4 - 0.0000000000001784 *dT**5 
    return day, month, year, day_of_week, GMST
N = 251
N4 = 5
calc_julian_day(N4, N)

2456177.5
2488222.0 68.0 4573.0 12.0 190.0 6.0
7.0 9.0 2012.0 5.0 29191.4428297696


In [9]:
import Minimizing
res1 = Minimizing.solve_navigation_task_SLSQP(data)
print(res1)
dt = res1.x[-1] / Constants.c
print(f'dt: {dt}')
import Transformations
lla = Transformations.ecef2lla(*res1.x[:-1])
print(lla)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 48929587592.090385
            Iterations: 86
            Function evaluations: 585
            Gradient evaluations: 86
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 48929587592.090385
       x: [ 2.828e+06  2.373e+06  3.724e+06 -3.456e+06]
     nit: 86
     jac: [-2.048e+04 -8.858e+04 -8.704e+04 -5.120e+02]
    nfev: 585
    njev: 86
dt: -0.011528917924381701
(45.479663799024976, 40.00324263089771, -1123275.200322042)


In [12]:
res2 = Minimizing.solve_navigation_task_LevMar(data)

print(res2)
dt = res2.x[-1] / Constants.c

print(f'dt: {dt}')
lla = Transformations.ecef2lla(*res2.x[:-1])
print(lla)
print(sum(res2.fun))

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 7.471e-04  1.012e-03  3.840e-04  1.216e-04]
           x: [ 2.775e+06  2.175e+06  5.571e+06 -2.038e+06]
        cost: 8.721114388533032e-07
         jac: [[-3.639e-02  1.346e-02 -3.854e-02  5.467e-02]
               [-3.720e-03 -2.761e-02 -5.720e-02  6.362e-02]
               [ 2.072e-02 -1.789e-02 -2.808e-02  3.919e-02]
               [-1.296e-02 -9.325e-03  1.519e-02 -2.205e-02]]
        grad: [-2.457e-05 -2.589e-05 -9.561e-05  1.176e-04]
  optimality: 0.00011758257695835634
 active_mask: [0 0 0 0]
        nfev: 211
        njev: None
dt: -0.006799469126686438
(57.83694369172598, 38.08445483012054, 230385.68521757185)
0.0022645259861857475


In [11]:
res3 = Minimizing.solve_navigation_task_GO(data)
print(res3)
dt = res3.x[-1] / Constants.c

print(f'dt: {dt}')
lla = Transformations.ecef2lla(*res3.x[:-1])
print(lla)

 message: Optimization terminated successfully.
 success: True
     fun: 1.3877787807814457e-17
       x: [ 2.775e+06  2.175e+06  5.571e+06 -2.038e+06]
     nit: 295
    nfev: 17765
dt: -0.006799469217355879
(57.83694333146158, 38.08445499698492, 230385.69636427402)
